In [1]:
import pandas as pd
import datetime
import numpy as np
import pickle
from scipy import spatial
import itertools
from scipy import signal
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import urllib2 
import urllib
import operator
import string
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.embed import components 
from bokeh.io import hplot

In [2]:
#READ IN THE YELP FILES to a pandas data frame
user = pd.read_csv("yelp_academic_dataset_user.csv", index_col = 'user_id')
print "user read-in complete"
print(user.shape)

business = pd.read_csv("yelp_academic_dataset_business.csv", na_filter=False, index_col = 'business_id')
print "business read-in complete"
print(business.shape)

reviews = pd.read_csv("yelp_academic_dataset_review.csv", index_col = 'review_id')
print "reviews read-in complete"
print(reviews.shape)

user read-in complete
(552339, 22)
business read-in complete
(77445, 97)
reviews read-in complete
(2225213, 9)


In [3]:
transformer = TfidfTransformer() #to do tfidf weighting on all of the bag_of_words vectors
nreturn = 10
nplot = 5

#reviews = pd.read_pickle('reviews_random')
reviews = pd.read_pickle('rev_and_business')
print reviews[0:5]

              business_id       date  day  month  stars  \
0  5UmKMjUEUNdYWqANhGckJw 2012-08-01    1      8      4   
1  5UmKMjUEUNdYWqANhGckJw 2014-02-13   13      2      5   
2  5UmKMjUEUNdYWqANhGckJw 2015-10-31   31     10      5   
3  WaHXyBwljbKNPmmJZn5j8Q 2012-08-01    1      8      3   
4  WaHXyBwljbKNPmmJZn5j8Q 2014-07-26   26      7      3   

                                          text_clean  year business_name  \
0  mr hoagie institution walking seem like throwb...  2012     Mr Hoagie   
1  excellent food superb customer service miss ma...  2014     Mr Hoagie   
2  yes place little dated opened weekend staff al...  2015     Mr Hoagie   
3                                          shop save  2012  Shop 'n Save   
4  living alone love deals meat also love faygo p...  2014  Shop 'n Save   

   business_stars          city state  
0             4.5    Dravosburg    PA  
1             4.5    Dravosburg    PA  
2             4.5    Dravosburg    PA  
3             3.0  West Miff

In [6]:
my_city = 'Las Vegas'
#my_city = 'Los Angeles'

reviews_total = []
reviews_subset = reviews[reviews['city'] == my_city]

begin_date = min(reviews_subset['date'])
end_date = max(reviews_subset['date'])

datelist = np.arange(begin_date, '2020-01-01', dtype='datetime64[D]')

next_date = begin_date

i=1
while next_date < end_date:
    next_date = next_date + pd.DateOffset(months=+3, days=0)
    datelist[i] = next_date
    i = i+1

datelist = datelist[0:i]
    
dlen = len(datelist)

print len(reviews_subset)
for i in range(0,dlen-1):
    this_month = reviews_subset[(reviews_subset['date'] > datelist[i]) & (reviews_subset['date'] <= datelist[i+1])]
    #if len(this_month) > 0:
    rtext = this_month['text_clean']
    if len(rtext) > 100:
        rtext = rtext[0:100]
    rtext_combined = string.join(rtext)
    reviews_total.append(rtext_combined)

print len(datelist)
print len(reviews_total)

#for i in range( 0, nrev ):
#    reviews_clean_subset.append(reviews_subset['text_clean'].iloc[i])

#mystring_subset = ''.join(reviews_clean_subset)

#reviews_total.append(mystring_subset)

861536
46
45


In [7]:
reviews_vec = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 200, ngram_range=(1,1)) 
reviews_features = reviews_vec.fit_transform(reviews_total)
reviews_tfidf = transformer.fit_transform(reviews_features)
reviews_features = reviews_features.toarray()
reviews_tfidf = reviews_tfidf.toarray()


reviews2_vec = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 200, ngram_range=(2,2)) 
reviews2_features = reviews2_vec.fit_transform(reviews_total)
reviews2_tfidf = transformer.fit_transform(reviews2_features)
reviews2_features = reviews2_features.toarray()
reviews2_tfidf = reviews2_tfidf.toarray()


reviews3_vec = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 200, ngram_range=(3,3)) 
reviews3_features = reviews3_vec.fit_transform(reviews_total)
reviews3_tfidf = transformer.fit_transform(reviews3_features)
reviews3_features = reviews3_features.toarray()
reviews3_tfidf = reviews3_tfidf.toarray()

vocab = reviews_vec.get_feature_names()
vocab2 = reviews2_vec.get_feature_names()
vocab3 = reviews3_vec.get_feature_names()

In [61]:
print vocab
print ''
print vocab2
print ''
print vocab3
print ''

[u'actually', u'also', u'always', u'amazing', u'another', u'anything', u'area', u'around', u'asked', u'away', u'awesome', u'back', u'bad', u'bar', u'bathroom', u'beautiful', u'bed', u'beds', u'bellagio', u'best', u'better', u'big', u'bit', u'buffet', u'called', u'came', u'casino', u'cheap', u'check', u'circus', u'clean', u'club', u'come', u'comfortable', u'cool', u'could', u'course', u'day', u'decor', u'definitely', u'desk', u'different', u'door', u'drink', u'drinks', u'eat', u'else', u'end', u'enough', u'even', u'ever', u'every', u'everything', u'expensive', u'experience', u'far', u'favorite', u'feel', u'find', u'first', u'floor', u'food', u'free', u'friendly', u'friends', u'front', u'fun', u'get', u'getting', u'give', u'go', u'going', u'good', u'got', u'great', u'hard', u'high', u'hot', u'hotel', u'hotels', u'hour', u'hours', u'however', u'huge', u'inside', u'kids', u'know', u'large', u'las', u'last', u'least', u'left', u'let', u'like', u'line', u'little', u'location', u'long', u'loo

In [8]:
i=1
dist = reviews_tfidf[i] #one row of word features
count_dict = dict(zip(vocab, dist))
sorted_count_dict = sorted(count_dict.items(), key=operator.itemgetter(1),reverse=True)
word, freq = zip(*sorted_count_dict)

dist2 = reviews2_tfidf[i] #one row of word features
count_dict2 = dict(zip(vocab2, dist2))
sorted_count_dict2 = sorted(count_dict2.items(), key=operator.itemgetter(1),reverse=True)
word2, freq2 = zip(*sorted_count_dict2)

dist3 = reviews3_tfidf[i] #one row of word features
count_dict3 = dict(zip(vocab3, dist3))
sorted_count_dict3 = sorted(count_dict3.items(), key=operator.itemgetter(1),reverse=True)
word3, freq3 = zip(*sorted_count_dict3)

ugram = word[0:nreturn]
bgram = word2[0:nreturn]
tgram = word3[0:nreturn]

result = pd.DataFrame({'unigram' : ugram, 'bigram' : bgram, 'trigram' : tgram})
print result

          bigram             trigram   unigram
0     black jack    really feel like      food
1     little bit    black jack table     great
2  stay bellagio     las vegas hotel     hotel
3      feel like  room circus circus    buffet
4      las vegas     hotel las vegas  bellagio
5     strip view       make sure get     rooms
6     great food   room smelled like     vegas
7     great deal    non smoking room       bit
8    vegas hotel     queen size beds      best
9     water show   wynn tower suites     table


In [11]:
for i in range(dlen-1):
    dist = reviews3_tfidf[i] #one row of word features
    count_dict = dict(zip(vocab3, dist))
    sorted_count_dict = sorted(count_dict.items(), key=operator.itemgetter(1),reverse=True)
    word, freq = zip(*sorted_count_dict)
    
    print datelist[i], map(lambda x: x.encode('ascii'), word[0:1]), freq[0:1]

2004-10-18 ['room circus circus'] (0.0,)
2005-01-18 ['really feel like'] (0.64942304510529991,)
2005-04-18 ['non smoking rooms'] (0.46200635954304642,)
2005-07-18 ['canyon ranch spa'] (0.61956495697565195,)
2005-10-18 ['first trip vegas'] (0.45389135251720991,)
2006-01-18 ['call front desk'] (0.35842778925714119,)
2006-04-18 ['floor ceiling windows'] (0.41006305999147891,)
2006-07-18 ['non smoking room'] (0.43417743855159024,)
2006-10-18 ['long story short'] (0.34376898325323169,)
2007-01-18 ['canyon ranch spa'] (0.32663235047853745,)
2007-04-18 ['love love love'] (0.70866568607141323,)
2007-07-18 ['canyon ranch spa'] (0.41609321558250478,)
2007-10-18 ['sunken living room'] (0.40469623903037633,)
2008-01-18 ['new york new'] (0.39670121221360399,)
2008-04-18 ['sunken living room'] (0.42810740130816893,)
2008-07-18 ['visit las vegas'] (0.22145580034457465,)
2008-10-18 ['new york new'] (0.29326921635828435,)
2009-01-18 ['hotel las vegas'] (0.24889001734579627,)
2009-04-18 ['first time sta

In [63]:
word_rate = {'datelist': datelist}
word_rate2 = {'datelist': datelist}
word_rate3 = {'datelist': datelist}

my_word_arr = ugram[0:nplot+1]
my_word2_arr = bgram[0:nplot+1]
my_word3_arr = tgram[0:nplot+1]

for j in range(0,len(my_word_arr)):
    for i in range(1,200):
        if vocab[i] == my_word_arr[j]:
            word_index = i
            break
    
    my_word_rate = []
    for i in range(dlen-1):
        dist = reviews_tfidf[i]
        my_word_rate.append(dist[word_index])

    word_rate[my_word_arr[j]] = my_word_rate
            
        
for j in range(0,len(my_word2_arr)):
    for i in range(1,200):
        if vocab2[i] == my_word2_arr[j]:
            word_index = i
            break
            
    my_word_rate = []
    for i in range(dlen-1):
        dist = reviews2_tfidf[i]
        my_word_rate.append(dist[word_index])

    word_rate2[my_word2_arr[j]] = my_word_rate

for j in range(0,len(my_word3_arr)):
    for i in range(1,200):
        if vocab3[i] == my_word3_arr[j]:
            word_index = i
            break

    my_word_rate = []
    for i in range(dlen-1):
        dist = reviews3_tfidf[i]
        my_word_rate.append(dist[word_index])

    word_rate3[my_word3_arr[j]] = my_word_rate



In [64]:
TOOLS = "pan,wheel_zoom,box_zoom,reset"
color_list = ['blue','red', 'green', 'orange', 'black']

plot1 = figure(tools=TOOLS,
              title='Frequently used words and phrases',
              x_axis_label='date',
              x_axis_type='datetime',
              y_axis_label = 'relative word frequency')

for j in range(nplot):
    plot1.line(datelist, word_rate[my_word_arr[j]], color=color_list[j], legend = my_word_arr[j])

plot1.legend.location = "top_right"


plot2 = figure(tools=TOOLS,
              title='Frequently used words and phrases',
              x_axis_label='date',
              x_axis_type='datetime',
              y_axis_label = 'relative word frequency')

for j in range(nplot):
    plot2.line(datelist, word_rate2[my_word2_arr[j]], color=color_list[j], legend = my_word2_arr[j])

plot2.legend.location = "top_right"

plot3 = figure(tools=TOOLS,
              title='Frequently used words and phrases',
              x_axis_label='date',
              x_axis_type='datetime',
              y_axis_label = 'relative word frequency')

for j in range(nplot):
    plot3.line(datelist, word_rate3[my_word3_arr[j]], color=color_list[j], legend = my_word3_arr[j])

plot3.legend.location = "top_right"

p = hplot(plot1, plot2, plot3)

script, div = components(p)

# show the results
show(p)

/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:38: BokehDeprecationWarning: bokeh.io.hplot was deprecated in Bokeh 0.12.0; please use bokeh.models.layouts.Row instead


In [55]:
city_list = reviews['city'].unique()
    
rev_by_city = reviews.groupby(['city']).count()
rev_by_city2 = rev_by_city[rev_by_city['state'] > 800]

city_list = []

for c in rev_by_city2.index:
    city_list.append(c)

for c in city_list:
    print c,',',reviews[reviews['city']==c].state.iloc[0]
    

#MontrMontr\xc3\xa9al is Montréal

Anthem , AZ
Apache Junction , AZ
Avondale , AZ
Belmont , NC
Buckeye , AZ
Carefree , AZ
Casa Grande , AZ
Cave Creek , AZ
Champaign , IL
Chandler , AZ
Charlotte , NC
Concord , NC
Dallas , TX
Edinburgh , EDH
Fitchburg , WI
Fort Mill , SC
Fountain Hills , AZ
Gilbert , AZ
Glendale , AZ
Goodyear , AZ
Henderson , NV
Homestead , PA
Karlsruhe , BW
Kitchener , ON
Las Vegas , NV
Laval , QC
Laveen , AZ
Litchfield Park , AZ
Madison , WI
Maricopa , AZ
Matthews , NC
Mesa , AZ
Middleton , WI
Monona , WI
Montreal , QC
Montréal , QC
North Las Vegas , NV
Paradise Valley , AZ
Peoria , AZ
Phoenix , AZ
Pineville , NC
Pittsburgh , PA
Queen Creek , AZ
San Tan Valley , AZ
Scottsdale , AZ
Sun City , AZ
Sun Prairie , WI
Surprise , AZ
Tempe , AZ
Tolleson , AZ
Urbana , IL
Verdun , QC
Waterloo , ON
Wickenburg , AZ
